Avaliando a generalização de algoritmos: Tarefa 4 da disciplina Reconhecimento de Padrões. 

Professor: Francisco de Assis Boldt 

Aluno: Diogo de Santana Candido

In [90]:
# Versão da Linguagem Python
from platform import python_version
print('Versão da Linguagem Python Usada Neste Jupyter Notebook:', python_version())

Versão da Linguagem Python Usada Neste Jupyter Notebook: 3.8.10


Foi escolhido o Datasets "Caesarian Section Classification Dataset" da UCI Marchine Learning Repository que disponibiliza dados coletados de grávidas com diferentes estados de saúde. Os seus metadados podem ser obtidos em https://archive.ics.uci.edu/ml/datasets/Caesarian+Section+Classification+Dataset#

Este trabalho tem por objetivo comparar os classificadores Logistic Regression e KNN do scikit-learn utilizando duas formas de avaliação.

Atributos:
@attribute 'Age' { 22,26,28,27,32,36,33,23,20,29,25,37,24,18,30,40,31,19,21,35,17,38 }
@attribute 'Delivery number' { 1,2,3,4 }
@attribute 'Delivery time' { 0,1,2 } -> {0 = timely , 1 = premature , 2 = latecomer}
@attribute 'Blood of Pressure' { 2,1,0 } -> {0 = low , 1 = normal , 2 = high }
@attribute 'Heart Problem' { 1,0 } -> {0 = apt, 1 = inept }

Rótulo:
@attribute Caesarian { 0,1 } -> {0 = No, 1 = Yes }

Carga do Dataset:

In [91]:
import pandas as pd
import urllib.request
from scipy.io.arff import loadarff
from io import StringIO

url = 'https://archive.ics.uci.edu/ml/machine-learning-databases/00472/caesarian.csv.arff'
resp = urllib.request.urlopen(url)
data, meta = loadarff(StringIO(resp.read().decode('utf-8')))

df = pd.DataFrame(data)

for i in df.columns:
    if df[i].dtype == 'object':
        df[i] = df[i].str.decode('UTF-8')

X, y = df.iloc[:, :5], df.iloc[:, -1]

X.shape, y.shape, set(y)

((80, 5), (80,), {'0', '1'})

1° Avaliação: Classifica com todo o Dataset e calcula a acurácia.

In [92]:
from sklearn.metrics import accuracy_score
from sklearn.model_selection import cross_validate, RepeatedKFold
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import LogisticRegression
import numpy as np

for classif in [KNeighborsClassifier, LogisticRegression]: 
    modelo = classif()
    modelo.fit(X, y)
    ypred = modelo.predict(X)
    print("Acurácia: ", accuracy_score(y, ypred), " -- Classificador", classif())

Acurácia:  0.775  -- Classificador KNeighborsClassifier()
Acurácia:  0.75  -- Classificador LogisticRegression()


2° Avaliação: Divide o Dataset em treino e teste. 

Utiliza os dados de treino para ajustar o modelo e os dados de teste para calcular a acurácia. 

In [93]:
from sklearn.model_selection import train_test_split

X_treino, X_teste, y_treino, y_teste = train_test_split(X, y, test_size=0.33, random_state=42)

for classif in [KNeighborsClassifier, LogisticRegression]:  
    modelo = classif()
    modelo.fit(X_treino, y_treino)
    ypred = modelo.predict(X_teste)
    print("Acurácia: ", accuracy_score(y_teste, ypred), " -- Classificador", classif())
    

Acurácia:  0.5555555555555556  -- Classificador KNeighborsClassifier()
Acurácia:  0.6666666666666666  -- Classificador LogisticRegression()


3° Avaliação: Realiza a validação cruzada e calcula a média da acurácia do total de repetições. 

In [94]:
from sklearn.metrics import make_scorer
kfold = RepeatedKFold(random_state=42) # n_repeats: default=10 e n_splits: default=5

for classif in [KNeighborsClassifier, LogisticRegression]: 
    scores = cross_validate(classif(), X, y, scoring=make_scorer(accuracy_score), cv=kfold)
    print("Acurácia: ", np.mean(scores['test_score']), " -- Classificador", classif())


Acurácia:  0.565  -- Classificador KNeighborsClassifier()
Acurácia:  0.605  -- Classificador LogisticRegression()


4° Avaliação: Realiza a padronização dos dados antes da validação cruzada e calcula a média da acurácia do total de repetições.

In [95]:
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()
X_pad = scaler.fit_transform(X)

for classif in [KNeighborsClassifier, LogisticRegression]: 
    scores = cross_validate(classif(), X_pad, y, scoring=make_scorer(accuracy_score), cv=kfold)
    print("Acurácia: ", np.mean(scores['test_score']), " -- Classificador", classif())

Acurácia:  0.665  -- Classificador KNeighborsClassifier()
Acurácia:  0.62375  -- Classificador LogisticRegression()


5° Avaliação: Realiza o autoajuste do modelo KNeighborsClassifier() determinando o melhor parâmetro e calcula a acurácia utilizando o Dataset com e sem padronização.   

In [96]:
from sklearn.model_selection import GridSearchCV
parametros = {'n_neighbors': [3,5,7]}

modelo = GridSearchCV(KNeighborsClassifier(), parametros, scoring=make_scorer(accuracy_score))
modelo.fit(X, y)
ypred = modelo.predict(X)
print("Acurácia sem padronização:", accuracy_score(y, ypred), "Parâmetro:", modelo.best_estimator_)


modelo.fit(X_pad, y)
ypred = modelo.predict(X_pad)
print("Acurácia com padronização:", accuracy_score(y, ypred), "Parâmetro:", modelo.best_estimator_)


Acurácia sem padronização: 0.7875 Parâmetro: KNeighborsClassifier(n_neighbors=3)
Acurácia com padronização: 0.75 Parâmetro: KNeighborsClassifier(n_neighbors=7)


6° Avaliação: Realiaza a validação cruzada do Pipeline com e sem o GridSearch e calcula a média da acurácia do total de repetições.

In [97]:
from sklearn.pipeline import Pipeline

GridSearchCV(KNeighborsClassifier(), parametros, scoring=make_scorer(accuracy_score))
parametros = {'n_neighbors': [3,5,7]}

modelo1 = Pipeline([
    ("padronização", StandardScaler()),
    ("gsknn", GridSearchCV(KNeighborsClassifier(), parametros, scoring=make_scorer(accuracy_score)))
])

scores = cross_validate(modelo1, X, y, scoring=make_scorer(accuracy_score), cv=kfold)
print("Acurácia: ", np.mean(scores['test_score']), "GridSearch dentro do Pipeline")

pipeline = Pipeline([
    ("padronização", StandardScaler()),
    ("knn", KNeighborsClassifier())
])

parametros = {'knn__n_neighbors': [3,5,7]}
modelo2 = GridSearchCV(pipeline, parametros, scoring=make_scorer(accuracy_score))
scores = cross_validate(modelo2, X, y, scoring=make_scorer(accuracy_score), cv=kfold)
print("Acurácia: ", np.mean(scores['test_score']), "GridSearch fora do Pipeline")





Acurácia:  0.65875 GridSearch dentro do Pipeline
Acurácia:  0.65 GridSearch fora do Pipeline
